# MLops

MLops est un ensemble de pratiques qui permettent d'automatiser les tâches de Machine Learning et l'optimisation de développement et déploiement des modèles. Dans ce tutoriel, on utilisera ``mlflow`` et ``argo workflows`` pour cela.

## 1- Mlflow

``MLflow`` est une plateforme qui permet de gérer des modèles de Machine Learning en enregistrant les paramètres et les métriques de performance. Il propose une interface utilisateur qui permet d'administrer et comparer les modèles facilement. Mlflow permet aussi de déployer les modèles via une API REST de manière simple et rapide. 

On commence par lancer mlflow sur Datalab. ***Il faut que la protection IP soit désactivée.***
<br> <img src="notebook-images/mlflow.jpg">

Ici, on va entrainer un modèle pour détecter les pizzas avec ``PyTorch``. Pour enregistrer les données avec mlflow, on suit les étapes suivantes:
- Au début, on indique à mlflow le nom de notre expérience. mlflow sauvegardera donc les données dans un dossier ayant le même nom. Pour cela, on utilise la commande ``mlflow.set_experiment(mlflow_experiment_name:str)``
- Avant de commencer la boucle for, on précise le contexte en utilisant la syntaxe ``with mlflow.start_run(run_name:str)``. Cela permet de suivre la progression de l'entrainement sur l'interface utilisateur mlflow ainsi que la création d'une instance sous le nom ``run_name`` où les données seront enregistrées. 
- Enregistrement des paramètres avec ``mlflow.log_parameter(param_name:str, param_value)``
- Enregistrement des métriques avec ``mlflow.log_metric(metric_name:str, metric_value)``
- Enregistrement du modèle avec ``mlflow.pytorch.log_model(model_name:str, model)``. mlflow est compatible avec plusieurs frameworks de Machine et Deep learning tels que PyTorch, Scikit-Learn, et TensorFlow.

Le code complet de la fonction main est disponible dans le fichier ``main.py`` (lequel on va utiliser par la suite)

In [ ]:
def train(lr, weight_decay, epochs, batch_size, mlflow_experiment_name, mlflow_run_name):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    device = torch.device("cpu")
    batch_size = batch_size
    data_root = "diffusion/pizza-not-pizza"

    dataset = DatasetGenerator(data_root)
    train_set_length = int(0.7 * len(dataset))
    test_set_length = len(dataset) - train_set_length
    #split the dataset 
    train_set, test_set = random_split(dataset, [train_set_length, test_set_length])
    train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
    test_loader = DataLoader(test_set, shuffle=True, batch_size=batch_size)
    class_num = 2
    ce_loss=nn.CrossEntropyLoss()

    #load model
    model = ResNet(class_num=class_num)
    model = model.to(device)

    #define optimizer
    opt_model = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)

    epoch = epochs
    model.train()
    torch.set_grad_enabled(True)
    #set experiment name
    mlflow.set_experiment(mlflow_experiment_name)
    #set context
    with mlflow.start_run(run_name=mlflow_run_name):
        for epo in range(1,epoch+1):
            correct_model = 0
            print("Epoch {}/{} \n".format(epo, epoch))
            with tqdm(total=len(train_loader), desc="Train") as pb:
                for batch_num, (img, img_label) in enumerate(train_loader):
                    opt_model.zero_grad()
                    img = img.to(device) 
                    img_label = img_label.to(device)       
                    outputs = model(img)
                    correct_model += (torch.argmax(outputs, dim=1)==img_label).sum().item()
                    loss = ce_loss(outputs, img_label)
                    loss.backward()
                    #loss_model.append(loss)
                    opt_model.step()
                    pb.update(1)
        train_accuracy = correct_model/len(train_set)
        val_accuracy = evaluate(model, test_loader, device)
        mlflow.log_param("learning rate", lr)
        mlflow.log_param("weight decay", weight_decay)
        mlflow.log_metric("valdiation accuracy", val_accuracy)
        mlflow.log_metric("training accuracy", train_accuracy)
        #mlflow.pytorch.log_model("model", model, code_paths=["model.py"])

Avant de lancer l'entrainement, on doit définir les hyperparamètres et les variables d'environnement nécessaires pour mlflow. Pour ce faire, on utilisera le script shell suivant:
```shell
#set environment variables
export MLFLOW_S3_ENDPOINT_URL='https://minio.lab.sspcloud.fr'
export MLFLOW_TRACKING_URI='https://user-mbenxsalha-561205.user.lab.sspcloud.fr/'
export MLFLOW_EXPERIMENT_NAME="Default"


# set hyper parameters
lr="0.01"
weight_decay="0.001"
epochs="1"
batch_size="64"
mlflow_experiment_name="Default"
mlflow_run_name="test"

root_path="/home/onyxia/work/"

python ${root_path}/main.py --lr lr --weight_decay weight_decay --epochs epochs --batch_size batch_size --mlflow_experiment_name mlflow_experiment_name --mlflow_run_name mlflow_run_name
```

***Il faut changer la variable ``MLFLOW_TRACKING_URI`` au lien de votre service mlflow***.

Ce script est enregistré dans le fichier ``local_mlflow.sh``

In [ ]:
#start training
!sh local_mlflow.sh

On peut aussi lancer l'entrainement dans un environnement virtuel. Pour cela, il faut mettre en place deux fichiers de configuration:
- ``conda.yaml``: dans ce fichier, on définit un environnement virtuel avec toutes les dépendances.
- ``MLProject``: dans ce fichier, on définit les paramètres et la commande pour lancer l'entrainement.

``conda.yaml``: 
```yaml
name: pizza-detector
channels:
  - defaults
dependencies:
  - python=3.8
  - pip
  - pip:
    - torch
    - torchvision
    - boto3==1.17.19
    - tqdm
    - argparse
    - mlflow
```

``MLProject``:
````
name: pizza-detector

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      remote_server_uri: {type: str, default: https://user-mbenxsalha-207868.user.lab.sspcloud.fr/}
      experiment_name: {type: str, default: Default}
      run_name: {type: str, default: default}
      lr: {type: float, default: 0.01}
      weight_decay: {type: float, default: 0.001}
      epochs: {type: int, default: 1}
      batch_size: {type: int, default: 64}
      
    command: "python main.py --lr {lr} --weight_decay {weight_decay} --epochs {epochs} --batch_size {batch_size} --mlflow_experiment_name {experiment_name} --mlflow_run_name {run_name}"
````

```shell
export MLFLOW_S3_ENDPOINT_URL='https://minio.lab.sspcloud.fr'
export MLFLOW_TRACKING_URI='https://user-mbenxsalha-561205.user.lab.sspcloud.fr/'
export MLFLOW_EXPERIMENT_NAME="Default"

run_name="default"

# set the hyper parameters
lr="0.01"
weight_decay="0.001"
epochs="1"
batch_size="64"

mlflow run . -P remote_server_uri=${MLFLOW_TRACKING_URI} \
-P experiment_name=${MLFLOW_EXPERIMENT_NAME} -P run_name=${run_name}
-P lr=${lr} -P weight_decay=${weight_decay} -P epochs=${epochs} -P batch_size=${batch_size}
```

In [ ]:
!sh remote_run.sh

## 2- Argo Workflow

Argo workflow est un workflow engine qui permet d'orchestrer des tâches de machine learning sur un cluster Kubernetes. Le workflow est défini par un graphe direct acyclique (DAG) où les noeuds définissent des tâches qui sont exécutée dans des conteneurs séparés parallèlement ou séquentiellement. Cela permet d'optimiser l'utilisation des ressources disponibles tout en accélérant le workflow remarquablement. 

Dans ce tutoriel, on va utiliser cet outil pour optimiser les hyperparamètres de notre modèle de classification de pizza. En l'occurence, on va entrainer plusieurs modèles avec des hyperparamètres différents et on va utiliser mlflow pour enregistrer les métriques de performance et comparer les différents modèles obtenus.

Pour commencer, on doit lancer un nouveau service ``argo-workflows`` sur Datalab.
<br> <img src="notebook-images/argo.jpg">

On installe le client argo workflow:

In [ ]:
! sudo sh scripts/argo_deb_install.sh

Ensuite, on configure un fichier ``workflow.yaml`` dans lequel on définit notre workflow. Ce workflow se décompose en trois parties:
- configuration des paramètres
- définition de DAG
- implémentation de DAG

***1- Configuration des paramètres***

On configure les paramètres de s3 pour pouvoir utiliser et enregistrer les données avec le serveur minio de SSPCloud. On configure aussi les paramètres de mlflow pour pouvoir communiquer avec le service mlflow. ``code-source-repo`` indique la repositoire github qui contient le code source de notre application. Ce repo doit contenir le fichier ``MLProject`` dans le chemin root. Le paramètre ``model-training-conf-list`` est une liste de dictionnaires contenant les différentes combinaisons d'hyperparamètres qu'on va tester.
```yaml
  arguments:
    parameters:
      - name: aws-access-id
        value: "changeme"
      - name: aws-secret-key
        value: "changeme"
      - name: aws-session-token
        value: "changeme"
      - name: aws-default-region
        value: "us-east-1"
      - name: aws-s3-endpoint
        value: "minio.lab.sspcloud.fr"
      - name: mlflow-tracking-uri
        value: 'https://user-mbenxsalha-561205.user.lab.sspcloud.fr/'
      - name: mlflow-experiment-name
        value: "Default"
      - name: mlflow-s3-url
        value: "https://minio.lab.sspcloud.fr"
      - name: code-source-repo
        value: "https://github.com/amine-bs/mlflow-argo.git"
      - name: model-training-conf-list
        value: |
          [
            { "lr": "0.01", "weight_decay": "0.001", "mlflow_run_name": "1"},
            { "lr": "0.01", "weight_decay": "0.0001", "mlflow_run_name": "2"},
            { "lr": "0.05", "weight_decay": "0.0001", "mlflow_run_name": "3"},
            { "lr": "0.05", "weight_decay": "0.001", "mlflow_run_name": "4"}

          ]
```

***2- Définition de DAG***

Ici, on définit les dépendances et les paramètres de chaque étape.
```yaml
    - name: main
      dag:
        tasks:
          # task 0: start pipeline
          - name: start-pipeline
            template: start-pipeline-wt
          # task 1: train model with given params
          - name: train-model-with-given-params
            dependencies: [ start-pipeline ]
            template: run-model-training-wt
            arguments:
              parameters:
                - name: lr
                  value: "{{item.lr}}"
                - name: weight_decay
                  value: "{{item.weight_decay}}"
                - name: mlflow_run_name
                  value: "{{item.mlflow_run_name}}"

              # pass the inputs to the step "withParam"
            withParam: "{{workflow.parameters.model-training-conf-list}}"
```

***3- Implémentation de DAG***

Ici, on définit les images ,les commandes et les variables d'environnement à utiliser pour chaque conteneur de chaque étape. Pour la première étape, on utilise une image de ``busybox`` pour initialiser le pipeline. Pour la deuxième étape, on utilise une simple image contenant ``conda``.
```yaml
    - name: start-pipeline-wt
      inputs:
      container:
        image: busybox
        command: [ sh, -c ]
        args: [ "echo start pipeline" ]

    # worker template for task-1 train model
    - name: run-model-training-wt
      inputs:
        parameters:
          - name: lr
          - name: weight_decay
          - name: mlflow_run_name
      container:
        image: liupengfei99/mlflow:latest
        command: [sh, -c]
        args: ["mlflow run $CODE_SOURCE_URI --version main -P remote_server_uri=$MLFLOW_TRACKING_URI -P experiment_name=$MLFLOW_EXPERIMENT_NAME -P lr={{inputs.parameters.lr}} -P weight_decay={{inputs.parameters.weight_decay}} -P mlflow_run_name={{inputs.parameters.mlflow_run_name}}"]
        env:
          - name: AWS_SECRET_ACCESS_KEY
            value: "{{workflow.parameters.aws-secret-key}}"
          - name: AWS_DEFAULT_REGION
            value: "{{workflow.parameters.aws-default-region}}"
          - name: AWS_S3_ENDPOINT
            value: "{{workflow.parameters.aws-s3-endpoint}}"
          - name: AWS_SESSION_TOKEN
            value: "{{workflow.parameters.aws-session-token}}"
          - name: AWS_ACCESS_KEY_ID
            value: "{{workflow.parameters.aws-access-id}}"
          - name: MLFLOW_TRACKING_URI
            value: "{{workflow.parameters.mlflow-tracking-uri}}"
          - name: MLFLOW_EXPERIMENT_NAME
            value: "{{workflow.parameters.mlflow-experiment-name}}"
          - name: MLFLOW_S3_ENDPOINT_URL
            value: "{{workflow.parameters.mlflow-s3-url}}"
          - name: CODE_SOURCE_URI
            value: "{{workflow.parameters.code-source-repo}}"
```

Enfin, on soumet notre workflow:

In [ ]:
!argo submit workflow.yaml

On peut voir les conteneurs qui viennent de se lancer avec ``kubectl``:

In [ ]:
!kubectl get pods

On peut aussi suivre la progression et les logs de chaque conteneur sur l'interface utilisateur d'Argo.
<img src="notebook-images/argo-ui.jpg">

On peut comparer les résultats obtenus de différents hyperparamètres en utilisant mlflow.
   <tr>
    <td> <img src="notebook-images/mlflow-ui.jpg" alt="Drawing" style="width: 600px;"/> </td>
    <td> <img src="notebook-images/mlflow-compare.jpg" alt="Drawing" style="width: 600px;"/> </td>
    </tr>